## Optimization code for PV microgrid:

In [1]:
### Initialization:

using JuMP, Gurobi, MATLAB


In [2]:
my_start =  1
my_end   = 96

# my_start = 65
# my_end   = 96

my_correct = 0 # Correction variable for loosening min voltage constraint (in case of infeasibility -> loose by 2.5%)

0

In [3]:
# for my_int = my_start:my_end

my_int = my_start

while my_int <= my_end

    #----------------------------------------------------------------------------------------------------
    ### Solver Definition:
    
    m = Model(solver=GurobiSolver(Presolve=1,TimeLimit=900))
        
    #----------------------------------------------------------------------------------------------------
    ### Loading Optimization Parameters:

    mat"generate_data($my_int)"
    
    vars = read_matfile("../../Data/Generated Data/5 - Optimization/scenarios/scen_"*dec(my_int)*".mat")

    interval  = Int64(jvalue(vars["interval"]))  # Current Interval (in day, for which we are optimizing)

    N_EMS   = Int64(jvalue(vars["N_EMS"]))   # num of remaining EMS   intervals (opt. executions)
    N_intra = Int64(jvalue(vars["N_intra"])) # num of remaining Intra intervals (opt. resolution)
    N_dies  = Int64(jvalue(vars["N_dies"]))  # num of diesel generators
    N_scen  = Int64(jvalue(vars["N_scen"]))  # num of different scenations
    N_firm  = Int64(jvalue(vars["N_firm"]))  # num of FIRM Margin Constraint Intervals

    # Upper and Lower bounds:
    P_bat_min = jvalue(vars["P_bat_min"])
    P_bat_max = jvalue(vars["P_bat_max"])
    eta_bat  = jvalue(vars["eta_bat"])
    SOC_min  = jvalue(vars["SOC_min"])
    SOC_max  = jvalue(vars["SOC_max"])
    SOC_init = jvalue(vars["SOC_init"])
    Cap_bat  = jvalue(vars["Cap_bat"])
    P_dies_max = jvalue(vars["P_dies_max"])
    P_dies_min = jvalue(vars["P_dies_min"])
    P_PV_inst = jvalue(vars["P_PV_inst"])

    delta_t = jvalue(vars["delta_t"]) # 1 min

    # f_min = -100000
    f_min   = jvalue(vars["f_min"])
    
    theta_i = jvalue(vars["theta_i"])
    theta_d = jvalue(vars["theta_d"])
    theta_b = jvalue(vars["theta_b"])
    theta_p = jvalue(vars["theta_p"])
    
    U_min_def = jvalue(vars["U_min_def"]) # - my_correct*0.025
    
    psi_min_i = jvalue(vars["psi_min_i"])
    psi_min_d = jvalue(vars["psi_min_d"])
    psi_min_p = jvalue(vars["psi_min_p"])
    psi_min_b = jvalue(vars["psi_min_b"])
    psi_min_c = jvalue(vars["psi_min_c"])    
    
    psi_loss_i = jvalue(vars["psi_loss_i"])
    psi_loss_d = jvalue(vars["psi_loss_d"])
    psi_loss_p = jvalue(vars["psi_loss_p"])
    psi_loss_b = jvalue(vars["psi_loss_b"])
    psi_loss_c = jvalue(vars["psi_loss_c"])
    
    marge_dies = jvalue(vars["marge_dies"])
    
    L_PV = jvalue(vars["L_PV"]) # generated PV scenarios
    L_C  = jvalue(vars["L_C"]) # generated load scenarios

    # Defining iterators:
    ts  = 1:N_EMS
    ps  = 1:N_intra
    ds  = 1:N_dies
    ss  = 1:N_scen
    m1s = 1:N_firm
    m2s = (N_firm+1):N_EMS
    
    #----------------------------------------------------------------------------------------------------
    ### Variable definitions:

    @variables m begin

        # Battery variables:
        P_bat_min <= P_bat_set[t=ts,p=ps]  <= P_bat_max # Battery setpoint for time t,p (decision var.)
        0 <= P_bat_cha[t=ts,p=ps] <= P_bat_max
        0 <= P_bat_dis[t=ts,p=ps] <= P_bat_max
        X_bat[t=ts,p=ps], Bin                           # Operating mode indicator (decision var.)
                                                        # -> P_bat_set = 2*(X_bat-0.5)*P_bat_abs
        SOC_min <= SOC_bat[t=ts,p=ps] <= SOC_max        # Obtained SOC for time t,p (solution var.)

        # PV Plant:
        0 <= P_PV_set[t=ts,p=ps]  <= P_PV_inst          # PV setpoint variable (decision var.)
        0 <= P_PV[t=ts,p=ps,s=ss] <= L_PV[t,p,s]        # Actual PV generation (solution var.)

        # Diesel Generators:
        ON_dies[t=ts,d=ds], Bin                         # Dies. connectin setpoint (decision var.)
        0 <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max  # Actual dies. generation  (solution var.)
        # P_dies_min <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max # Actual dies. generation  (solution var.)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Variable Initialization (from previous optimization)
    
    if my_int>1

        vars = read_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(my_int-1)*".mat")

        P_bat_set_pre = jvalue(vars["P_bat_set"])[2:N_EMS,ps]
        SOC_bat_pre   = jvalue(vars["SOC_bat"])[2:N_EMS,ps]
        P_PV_set_pre  = jvalue(vars["P_PV_set"])[2:N_EMS,ps]
        P_PV_pre      = jvalue(vars["P_PV"])[2:N_EMS,ps,ss]
        ON_dies_pre   = jvalue(vars["ON_dies"])[2:N_EMS,ds]
        P_dies_pre    = jvalue(vars["P_dies"])[2:N_EMS,ps,ss,ds]
        P_bat_cha_pre = jvalue(vars["P_bat_cha"])[2:N_EMS,ps]
        P_bat_dis_pre = jvalue(vars["P_bat_dis"])[2:N_EMS,ps]
        X_bat_pre     = jvalue(vars["X_bat"])[2:N_EMS,ps]

        setvalue(P_bat_set[1:(N_EMS-1),ps],P_bat_set_pre)
        setvalue(SOC_bat[1:(N_EMS-1),ps],SOC_bat_pre)
        setvalue(P_PV_set[1:(N_EMS-1),ps],P_PV_set_pre)
        setvalue(P_PV[1:(N_EMS-1),ps,ss],P_PV_pre)
        setvalue(ON_dies[1:(N_EMS-1),ds],ON_dies_pre)
        setvalue(P_dies[1:(N_EMS-1),ps,ss,ds],P_dies_pre)
        setvalue(P_bat_cha[1:(N_EMS-1),ps],P_bat_cha_pre)
        setvalue(P_bat_dis[1:(N_EMS-1),ps],P_bat_dis_pre)
        setvalue(X_bat[1:(N_EMS-1),ps],X_bat_pre)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Constraints introduction:
    
    @constraints m begin

        # Constraints:
        #
        # A1 - Diesel power limitation
        # A2 - Diesel power limitation
        # B1 - Power balance equation
        # C1 - PV setpoint limitation
        # D1 - Bat. setpoint equation
        # D2 - Charging    mode const.
        # D3 - Discharging mode const.
        # E1 - FIRM    Diesel reserve margin (separated over scenarios)
        # E2 - RELAXED Diesel reserve margin (mean over scenarios)
        # F1 - 
        # F2 - SOC limitations
        # F3 - 
        # G1 - Minimum frequency limitation
        # X1 - Minimum voltage relaxed limitation
        # X2 - Minimum voltage firm    limitation

        A1[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] <= ON_dies[t,d]*(P_dies_max)
        A2[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] >= ON_dies[t,d]*(P_dies_min)

        # B1[t=ts,p=ps,s=ss], (P_PV[t,p,s] + sum(P_dies[t,p,s,d] for d=ds) + P_bat_set[t,p] - L_C[t,p,s]) == 0
        B1[t=ts,p=ps,s=ss], (P_PV[t,p,s]*(1-psi_loss_p) + sum((P_dies[t,p,s,d] - psi_loss_d*1e3*ON_dies[t,d]) for d=ds) + P_bat_set[t,p]*(1-psi_loss_b) - L_C[t,p,s]*(1+psi_loss_c) - psi_loss_i*1e3) == 0
                
        C1[t=ts,p=ps,s=ss],  P_PV[t,p,s] <= P_PV_set[t,p]

        D1[t=ts,p=ps], P_bat_set[t,p] == P_bat_cha[t,p] - P_bat_dis[t,p]
        D2[t=ts,p=ps], P_bat_cha[t,p] <= P_bat_max *    X_bat[t,p]
        D3[t=ts,p=ps], P_bat_dis[t,p] <= P_bat_max * (1-X_bat[t,p])

        # E1[t=ts,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E1[t=m1s,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E2[t=m2s,p=ps],  sum(P_dies[t,p,s,d] for d=ds,s=ss)/N_scen <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies

        F1[t=1,p=1],               SOC_bat[t,p] == SOC_init             - P_bat_set[t,p]*delta_t/Cap_bat
        F2[t=2:N_EMS,p=1],         SOC_bat[t,p] == SOC_bat[t-1,N_intra] - P_bat_set[t,p]*delta_t/Cap_bat
        F3[t=1:N_EMS,p=2:N_intra], SOC_bat[t,p] == SOC_bat[t,p-1]       - P_bat_set[t,p]*delta_t/Cap_bat

        G1[t=ts,p=ps], f_min <= theta_i + theta_d*sum(ON_dies[t,d] for d=ds) + theta_b*P_bat_set[t,p] + theta_p*P_PV_set[t,p]
        
        # X1[t=ts ,p=ps,s=ss], psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*P_PV[t,p,s] + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*L_C[t,p,s] >= U_min_def
        # X1[t=m1s,p=ps,s=ss], psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*P_PV[t,p,s]                      + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*L_C[t,p,s]                      >= U_min_def
        # X2[t=m2s,p=ps],      psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*sum(P_PV[t,p,s] for s=ss)/N_scen + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*sum(L_C[t,p,s] for s=ss)/N_scen >= U_min_def
        
    end

    #----------------------------------------------------------------------------------------------------
    ### Objective function definition:
    
    @objective(m, Max, sum((sum(P_PV[t,p,s] for s=ss)-N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p])) for t=ts,p=ps));
    
    #----------------------------------------------------------------------------------------------------
    ### Solving Optimization:
    status = solve(m)

    #----------------------------------------------------------------------------------------------------
    ### Saving Data:
    
    # If Solved => Write Down the Results
    if string(status)!="Infeasible"
        write_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(interval)*".mat"; 
            status    = string(status),
            interval  = interval,
            P_bat_set = getvalue(P_bat_set[:,:]), 
            SOC_bat   = getvalue(SOC_bat[:,:]),
            P_PV_set  = getvalue(P_PV_set[:,:]),
            P_PV      = getvalue(P_PV[:,:,:]),
            ON_dies   = getvalue(ON_dies[:,:]),
            P_dies    = getvalue(P_dies[:,:,:,:]),
            P_bat_cha = getvalue(P_bat_cha[:,:]),
            P_bat_dis = getvalue(P_bat_dis[:,:]),
            X_bat     = getvalue(X_bat[:,:]))    
        my_correct = 0
    # Usual Culprit for Infeas is Min Volt -> We loosen the constraint a little!!!
    else 
        my_int-=1
        if my_correct<2
            my_correct+=1
            println("Interval "*dec(my_int+1)*" is Infeasible !!!")
            println("Loosening Min Voltage Constraint !!!")
        else
            println("Loosening did not work -> Terminating Optimization!!!")
            break
        end
    end
    
    # IJulia.clear_output(wait=false)
    println("Current progress: "*dec(my_int)*"/"*dec(N_EMS))
    
    my_int+=1
    
end

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]
Presolve removed 8647 rows and 5104 columns
Presolve time: 1.48s
Presolved: 187913 rows, 98288 columns, 655848 nonzeros
Variable types: 95696 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   29578    1.7950782e+07   1.138956e+06   0.000000e+00      5s
   59652    1.7919194e+07   6.450577e+05   0.000000e+00     10s
   85163    1.7851648e+07   4.744418e+05   0.000000e+00     15s
  108185    1.7687611e+07   2.720978e+05   0.000000e+00     20s
  131233    1.7419815e+07   1.220690e+05   0.000000e+00     25s
  154656    1.5711900e+07   1.800444e+05   0.000000e+00     30s
  166709  

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 131.738625690

Presolve removed 8656 rows and 5107 columns
Presolve time: 1.66s
Presolved: 187904 rows, 98285 columns, 655836 nonzeros
Variable types: 95693 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    4918    1.8060192e+07   8.261447e+06   0.000000e+00      5s
   42377    1.7910686e+07   8.789355e+05   0.000000e+00     10s
   68024    1.7878084e+07   6.109322e+05   0.000000e+00     15s
   87129    1.7832246e+07   4.338338e+05   0.000000e+00     20s
  104079    1.7739742e+07   3.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 693.363906201

Presolve removed 8667 rows and 5114 columns
Presolve time: 1.81s
Presolved: 187893 rows, 98278 columns, 655818 nonzeros
Variable types: 95686 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   19067    1.7795087e+07   1.722941e+06   0.000000e+00      5s
   51418    1.7759283e+07   7.333308e+05   0.000000e+00     10s
   76246    1.7725753e+07   5.342200e+05   0.000000e+00     15s
   99773    1.7623545e+07   3.607026e+05   0.000000e+00     20s
  122531    1.7445500e+07   2.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 120.112772201

Presolve removed 8683 rows and 5140 columns
Presolve time: 1.62s
Presolved: 187877 rows, 98252 columns, 655776 nonzeros
Variable types: 95660 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   23969    1.8081135e+07   1.263185e+06   0.000000e+00      5s
   52887    1.8065927e+07   7.133598e+05   0.000000e+00     10s
   73273    1.8034560e+07   5.445728e+05   0.000000e+00     15s
   96022    1.7940777e+07   3.675050e+05   0.000000e+00     20s
  117706    1.7764337e+07   2.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 673.703699693

Presolve removed 8690 rows and 5141 columns
Presolve time: 1.67s
Presolved: 187870 rows, 98251 columns, 655768 nonzeros
Variable types: 95659 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   23815    1.7857344e+07   1.252222e+06   0.000000e+00      5s
   52625    1.7833775e+07   7.317998e+05   0.000000e+00     10s
   78777    1.7793366e+07   5.192739e+05   0.000000e+00     15s
  102618    1.7688275e+07   3.085733e+05   0.000000e+00     20s
  124556    1.7485144e+07   1.

...

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 501.014254425

Presolve removed 8671 rows and 5133 columns
Presolve time: 1.54s
Presolved: 187889 rows, 98259 columns, 655795 nonzeros
Variable types: 95667 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8024805e+07   8.967562e+06   0.000000e+00      6s
   45221    1.7868096e+07   7.460629e+05   0.000000e+00     10s
   72516    1.7832260e+07   5.493236e+05   0.000000e+00     15s
   95499    1.7746489e+07   3.679047e+05   0.000000e+00     20s
  122382    1.7487506e+07   1.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [2e-01, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 1222.918727844

Presolve removed 8668 rows and 5120 columns
Presolve time: 1.64s
Presolved: 187892 rows, 98272 columns, 655811 nonzeros
Variable types: 95680 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8198256e+07   8.849794e+06   0.000000e+00      6s
   45919    1.8039760e+07   7.756677e+05   0.000000e+00     10s
   74042    1.7995724e+07   5.537592e+05   0.000000e+00     15s
   98198    1.7889066e+07   3.292547e+05   0.000000e+00     20s
  122870    1.7658059e+07   1

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 379.490092548

Presolve removed 8667 rows and 5115 columns
Presolve time: 1.59s
Presolved: 187893 rows, 98277 columns, 655817 nonzeros
Variable types: 95685 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8150551e+07   8.887564e+06   0.000000e+00      5s
   49463    1.7995631e+07   7.548411e+05   0.000000e+00     10s
   75342    1.7955514e+07   5.204357e+05   0.000000e+00     15s
   98109    1.7866181e+07   3.503009e+05   0.000000e+00     20s
  122138    1.7647013e+07   1.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 86.934728292

Presolve removed 8666 rows and 5119 columns
Presolve time: 1.50s
Presolved: 187894 rows, 98273 columns, 655826 nonzeros
Variable types: 95681 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8150336e+07   8.754243e+06   0.000000e+00      6s
   47189    1.7979648e+07   8.013179e+05   0.000000e+00     10s
   74534    1.7936267e+07   5.777285e+05   0.000000e+00     15s
   99728    1.7827150e+07   3.973734e+05   0.000000e+00     20s
  125554    1.7569383e+07   1.8

Excessive output truncated after 526248 bytes.